# Chapter 40: Importlib Basics

This notebook covers the core functionality of Python's `importlib` module -- the programmatic interface to the import system. You will learn how to import modules by string name, reload modules, and inspect the standard attributes that every module carries.

## Key Concepts
- **`importlib.import_module`**: Import a module using a string name instead of the `import` statement
- **`importlib.reload`**: Re-import an already-loaded module to pick up changes
- **Module attributes**: Standard attributes like `__name__`, `__file__`, and `__package__`
- **`__spec__`**: The `ModuleSpec` object that describes how a module was found and loaded
- **Submodule imports**: Importing dotted module paths like `os.path`

## Section 1: Importing Modules by Name

The `importlib.import_module` function lets you import a module when you have its name as a string. This is essential for plugin systems, dynamic loading, and any scenario where the module name is not known until runtime.

In [ ]:
import importlib
import types

# Import a module by string name
math_mod: types.ModuleType = importlib.import_module("math")

print(f"Module: {math_mod}")
print(f"pi = {math_mod.pi}")
print(f"sqrt(16) = {math_mod.sqrt(16)}")

# This is equivalent to 'import math'
import math
print(f"\nSame object? {math_mod is math}")

In [ ]:
import importlib
import types

# Dynamic import from a list of module names
module_names: list[str] = ["json", "math", "os"]

for name in module_names:
    mod: types.ModuleType = importlib.import_module(name)
    attr_count: int = len([a for a in dir(mod) if not a.startswith("_")])
    print(f"{name:>6}: {attr_count} public attributes")

## Section 2: Importing Submodules

You can import submodules (dotted names like `os.path`) by passing the full dotted path to `import_module`. This works for any package that contains submodules.

In [ ]:
import importlib
import types

# Import a submodule using its dotted path
path_mod: types.ModuleType = importlib.import_module("os.path")

print(f"Module: {path_mod}")
print(f"Has 'join': {hasattr(path_mod, 'join')}")
print(f"Has 'exists': {hasattr(path_mod, 'exists')}")

# Demonstrate it works
result: str = path_mod.join("/usr", "local", "bin")
print(f"\nos.path.join('/usr', 'local', 'bin') = {result}")

In [ ]:
import importlib
import types

# Import submodules from the email package
email_mime: types.ModuleType = importlib.import_module("email.mime.text")

print(f"Module: {email_mime}")
print(f"Has 'MIMEText': {hasattr(email_mime, 'MIMEText')}")

# You can also use the 'package' parameter for relative-style imports
# This imports 'email.mime.base' using 'email.mime' as the package anchor
base_mod: types.ModuleType = importlib.import_module(".base", package="email.mime")
print(f"\nRelative import: {base_mod}")
print(f"Has 'MIMEBase': {hasattr(base_mod, 'MIMEBase')}")

## Section 3: Reloading Modules

Python caches imported modules in `sys.modules`. If you modify a module's source code at runtime, calling `importlib.reload` re-executes the module's code and updates the existing module object in place.

In [ ]:
import importlib
import json
import types

# Store the original module identity
original_id: int = id(json)

# Reload the json module
reloaded: types.ModuleType = importlib.reload(json)

# reload returns the same module object (updated in place)
print(f"Original id: {original_id}")
print(f"Reloaded id: {id(reloaded)}")
print(f"Same object: {reloaded is json}")
print(f"Still works:  json.dumps([1, 2, 3]) = {json.dumps([1, 2, 3])}")

In [ ]:
import importlib
import sys
import types
import tempfile
import os

# Demonstrate reload with a temporary module
# Create a temporary module file
tmp_dir: str = tempfile.mkdtemp()
mod_path: str = os.path.join(tmp_dir, "demo_reload.py")

# Write version 1
with open(mod_path, "w") as f:
    f.write("VERSION: int = 1\n")

# Add tmp_dir to sys.path so we can import it
sys.path.insert(0, tmp_dir)

try:
    mod: types.ModuleType = importlib.import_module("demo_reload")
    print(f"Version after first import: {mod.VERSION}")

    # Write version 2
    with open(mod_path, "w") as f:
        f.write("VERSION: int = 2\n")

    # Without reload, we still see version 1
    mod2: types.ModuleType = importlib.import_module("demo_reload")
    print(f"Version after re-import (cached): {mod2.VERSION}")

    # With reload, we pick up version 2
    importlib.reload(mod)
    print(f"Version after reload: {mod.VERSION}")
finally:
    # Clean up
    sys.path.remove(tmp_dir)
    if "demo_reload" in sys.modules:
        del sys.modules["demo_reload"]
    os.remove(mod_path)
    os.rmdir(tmp_dir)

## Section 4: Standard Module Attributes

Every module in Python carries several standard attributes that describe its identity and location. These are set automatically by the import system.

In [ ]:
import json

# __name__: The fully qualified module name
print(f"__name__:    {json.__name__}")

# __file__: The path to the source file (if applicable)
print(f"__file__:    {json.__file__}")

# __package__: The package this module belongs to
print(f"__package__: {json.__package__}")

# __loader__: The loader that loaded this module
print(f"__loader__:  {json.__loader__}")

# __doc__: The module docstring (first few characters)
doc_preview: str = (json.__doc__ or "")[:80]
print(f"__doc__:     {doc_preview}...")

In [ ]:
import os
import math
import email.mime.text

# Compare attributes across different module types
modules: list[str] = ["os", "math", "email.mime.text"]

for name in modules:
    mod = __import__(name, fromlist=[""]) if "." in name else __import__(name)
    if "." in name:
        import importlib
        mod = importlib.import_module(name)

    has_file: bool = hasattr(mod, "__file__") and mod.__file__ is not None
    print(f"{name}:")
    print(f"  __name__    = {mod.__name__}")
    print(f"  __package__ = {mod.__package__}")
    print(f"  has __file__= {has_file}")
    print()

## Section 5: The `__spec__` Attribute

Every module has a `__spec__` attribute (a `ModuleSpec` object) that stores metadata about how the module was found and loaded. This was introduced in Python 3.4 and is the modern way to introspect the import system.

In [ ]:
import json
import importlib.machinery

# Access the module spec
spec: importlib.machinery.ModuleSpec | None = json.__spec__

print(f"spec is not None: {spec is not None}")
print(f"spec type: {type(spec).__name__}")
print()

if spec is not None:
    print(f"name:           {spec.name}")
    print(f"origin:         {spec.origin}")
    print(f"loader:         {spec.loader}")
    print(f"parent:         {spec.parent}")
    print(f"has_location:   {spec.has_location}")
    print(f"submodule_search_locations: {spec.submodule_search_locations}")

In [ ]:
import importlib

# Compare __spec__ for a package vs. a simple module
email_mod = importlib.import_module("email")
json_mod = importlib.import_module("json")

for mod in [email_mod, json_mod]:
    spec = mod.__spec__
    if spec is not None:
        is_package: bool = spec.submodule_search_locations is not None
        print(f"{spec.name}:")
        print(f"  origin:     {spec.origin}")
        print(f"  is package: {is_package}")
        print(f"  parent:     {spec.parent!r}")
        print()

## Section 6: Checking Module Existence

You can use `importlib.util.find_spec` to check whether a module can be imported without actually importing it. This is safer than catching `ImportError`.

In [ ]:
import importlib.util

# Check if various modules exist
module_names: list[str] = ["json", "numpy", "os.path", "nonexistent_module_xyz"]

for name in module_names:
    spec = importlib.util.find_spec(name)
    exists: bool = spec is not None
    print(f"{name:>30}: {'found' if exists else 'NOT found'}")

In [ ]:
import importlib.util
import types

def safe_import(name: str) -> types.ModuleType | None:
    """Import a module by name, returning None if not found."""
    spec = importlib.util.find_spec(name)
    if spec is None:
        return None
    return importlib.import_module(name)

# Try to import modules that may or may not exist
for name in ["json", "nonexistent_package_abc"]:
    mod: types.ModuleType | None = safe_import(name)
    if mod is not None:
        print(f"{name}: imported successfully (type={type(mod).__name__})")
    else:
        print(f"{name}: not available")

## Section 7: Practical Patterns

Common patterns that use `importlib` in real applications, including plugin loading and conditional imports.

In [ ]:
import importlib
import importlib.util
from typing import Any

def load_serializer(format_name: str) -> Any:
    """Load a serializer module by format name.

    Simulates a plugin system where the format name
    determines which module to load.
    """
    format_to_module: dict[str, str] = {
        "json": "json",
        "csv": "csv",
        "xml": "xml.etree.ElementTree",
    }

    module_name: str | None = format_to_module.get(format_name)
    if module_name is None:
        print(f"  Unknown format: {format_name}")
        return None

    if importlib.util.find_spec(module_name) is None:
        print(f"  Module {module_name} not available")
        return None

    return importlib.import_module(module_name)

# Test the plugin loader
for fmt in ["json", "csv", "xml", "yaml"]:
    print(f"Loading '{fmt}':")
    mod = load_serializer(fmt)
    if mod is not None:
        print(f"  Loaded: {mod.__name__}")

In [ ]:
import importlib
import importlib.util
import types

def get_module_info(name: str) -> dict[str, str | bool | None]:
    """Gather comprehensive information about a module."""
    info: dict[str, str | bool | None] = {"name": name}

    spec = importlib.util.find_spec(name)
    if spec is None:
        info["exists"] = False
        return info

    info["exists"] = True
    info["origin"] = spec.origin
    info["is_package"] = spec.submodule_search_locations is not None
    info["has_location"] = spec.has_location
    info["parent"] = spec.parent or None

    return info

# Inspect several modules
for name in ["json", "os", "os.path", "email", "math"]:
    info: dict[str, str | bool | None] = get_module_info(name)
    print(f"{name}:")
    for key, value in info.items():
        if key != "name":
            print(f"  {key}: {value}")
    print()

## Summary

### Core Functions
- **`importlib.import_module(name)`**: Import a module by string name, equivalent to `import name`
- **`importlib.import_module(name, package)`**: Import using relative dotted paths with a package anchor
- **`importlib.reload(module)`**: Re-execute a module's code and update the existing module object
- **`importlib.util.find_spec(name)`**: Check if a module exists without importing it

### Standard Module Attributes
- **`__name__`**: Fully qualified module name (e.g., `"json"`, `"os.path"`)
- **`__file__`**: Path to the module's source file (may be `None` for built-in modules)
- **`__package__`**: The package this module belongs to (empty string for top-level modules)
- **`__loader__`**: The loader object that loaded the module
- **`__spec__`**: A `ModuleSpec` with metadata about how the module was found and loaded

### ModuleSpec Attributes
- **`spec.name`**: The module's name
- **`spec.origin`**: Where the module was loaded from (file path or built-in)
- **`spec.loader`**: The loader responsible for this module
- **`spec.parent`**: The parent package name
- **`spec.submodule_search_locations`**: Not `None` if this module is a package

### Important Notes
- `import_module` returns the same cached object as a regular `import` statement
- `reload` updates the module in place -- the object identity (`id()`) stays the same
- `find_spec` returns `None` for modules that cannot be found
- Built-in modules (like `math`) may not have a `__file__` attribute